<a href="https://colab.research.google.com/github/sunshineluyao/ESG_with_benchmark/blob/main/Categorization/ESG_Categorization_Jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
### import packages
import pandas as pd
import numpy as np
### show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [43]:
### Import data
df_Jan= pd.read_csv("https://raw.githubusercontent.com/sunshineluyao/ESG_with_benchmark/main/rawdata/SP1500_Jan.csv")
df_Jan.head()

,Symbol,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price
0,A,"Agilent Technologies, Inc.",NYSE,Healthcare,"26,461.70",0.5,309.7,"26,461.70",27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1
1,AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,"12,280.80",-14.7,428.2,"12,280.80",-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6
2,AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,"3,835.10",12.1,66.7,"3,835.10",36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0
3,AAOI,"Applied Optoelectronics, Inc.",NASDAQ,Technology,239,13.4,20.1,239,-23.0,-4.9,200.4,NaN,NaN,NaN,@NA,239,11.9,251.5,12.5,243.9,12.1,243.9,12.1,243.9,12.1,244.5,12.2,250.1,12.4,255.5,12.7,262.8,13.1,260.2,12.9,260.2,12.9,260.2,12.9,272.4,13.5,278.9,13.9,313.9,15.6,288.6,14.3,284.1,14.1,284.1,14.1,284.1,14.1,284.1,14.1,275.6,13.7,286.7,14.3,286.3,14.2,280.9,14,280.9,14,280.9,14,266.8,13.3,260.4,12.9,240,11.9,234.4,11.7,227.4,11.3,20.4,1.0
4,AAON,"AAON, Inc.",NASDAQ,Industrials,"2,574.20",5.5,52.1,"2,574.20",41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52

In [44]:
df_Jan=df_Jan.dropna()

In [45]:
df_Jan=df_Jan.set_index('Symbol',drop=True)
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price
Symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,"Agilent Technologies, Inc.",NYSE,Healthcare,"26,461.70",0.5,309.7,"26,461.70",27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,"12,280.80",-14.7,428.2,"12,280.80",-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,"3,835.10",12.1,66.7,"3,835.10",36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0
AAON,"AAON, Inc.",NASDAQ,Industrials,"2,574.20",5.5,52.1,"2,574.20",41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,"11,092.60",26.7,69.2,"11,092.60",1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",

In [46]:
df_Jan.index.name=None
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price
A,"Agilent Technologies, Inc.",NYSE,Healthcare,"26,461.70",0.5,309.7,"26,461.70",27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,"12,280.80",-14.7,428.2,"12,280.80",-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,"3,835.10",12.1,66.7,"3,835.10",36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0
AAON,"AAON, Inc.",NASDAQ,Industrials,"2,574.20",5.5,52.1,"2,574.20",41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,"11,092.60",26.7,69.2,"11,092.60",1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",159.4,"11,041.30",159.4,"10,859.90",156.8,"10,731.10",154.9,"10,607.80",153.2,"10,587

In [47]:
### formating data so that it can be calculated
df_Jan[['Size','Common Shares Outstdg','MktVal Co']]=df_Jan[['Size','Common Shares Outstdg','MktVal Co']].apply(lambda x: x.str.replace(',', ''), axis=1)
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.70,0.5,309.7,26461.70,27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.80,-14.7,428.2,12280.80,-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.10,12.1,66.7,3835.10,36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.20,5.5,52.1,2574.20,41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.60,26.7,69.2,11092.60,1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",159.4,"11,041.30",159.4,"10,859.90",156.8,"10,731.10",154.9,"10,607.80",153.2,"10,587.70",152.9,"10,320.30",149,"10

In [48]:
df_Jan['ESG_Index'] = np.where(df_Jan['MSCI ESG Rtg Overall Company Rating'].isin(['A','AA','AAA']), 'High','Low')
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price,ESG_Index
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.70,0.5,309.7,26461.70,27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1,High
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.80,-14.7,428.2,12280.80,-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6,Low
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.10,12.1,66.7,3835.10,36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0,High
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.20,5.5,52.1,2574.20,41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4,High
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.60,26.7,69.2,11092.60,1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",159.4,"11,041.30",159.4,"10,859.90",156.8,"10,731.10",154.9,"10,607.80",153.2,"10,587.

In [49]:
df_Jan[['Size', 'Book Value Per Shr Tangble','Common Shares Outstdg','MktVal Co','Momentum']] = df_Jan[['Size', 'Book Value Per Shr Tangble','Common Shares Outstdg','MktVal Co','Momentum']].astype('float')
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price,ESG_Index
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.7,0.5,309.7,26461.7,27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1,High
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.8,-14.7,428.2,12280.8,-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6,Low
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.1,12.1,66.7,3835.1,36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0,High
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.2,5.5,52.1,2574.2,41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4,High
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.6,26.7,69.2,11092.6,1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",159.4,"11,041.30",159.4,"10,859.90",156.8,"10,731.10",154.9,"10,607.80",153.2,"10,587.70",152.9,

In [50]:
### Generate the new variable:BM_Ratio
df_Jan['BM_Ratio'] = df_Jan['Book Value Per Shr Tangble']*df_Jan['Common Shares Outstdg']/df_Jan['MktVal Co']
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price,ESG_Index,BM_Ratio
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.7,0.5,309.7,26461.7,27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1,High,0.005852
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.8,-14.7,428.2,12280.8,-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6,Low,-0.512551
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.1,12.1,66.7,3835.1,36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0,High,0.210443
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.2,5.5,52.1,2574.2,41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4,High,0.111316
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.6,26.7,69.2,11092.6,1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",159.4,"11,041.30",159.4,"10,859.90",156.8,"10,731.

In [51]:
df_Jan.count()

Name                                   1426
Stock Exchange                         1426
Company Main Business                  1426
Size                                   1426
Book Value Per Shr Tangble             1426
Common Shares Outstdg                  1426
MktVal Co                              1426
Momentum                               1426
Monthly Return                         1426
Sales                                  1426
MSCI ESG Rtg Score Environmental       1426
MSCI ESG Rtg Score Governance          1426
MSCI ESG Rtg Score Social              1426
MSCI ESG Rtg Overall Company Rating    1426
1 MktVal Co                            1426
1 Closing Price                        1426
2 MktVal Co                            1426
2 Closing Price                        1426
3 MktVal Co                            1426
3 Closing Price                        1426
4 MktVal Co                            1426
4 Closing Price                        1426
5 MktVal Co                     

In [52]:
### Generate the categories by quintile cut of size
df_Jan['Size_Index']=pd.qcut(df_Jan['Size'],5,labels=np.arange(1,6,1))
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price,ESG_Index,BM_Ratio,Size_Index
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.7,0.5,309.7,26461.7,27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1,High,0.005852,5
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.8,-14.7,428.2,12280.8,-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6,Low,-0.512551,4
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.1,12.1,66.7,3835.1,36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0,High,0.210443,3
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.2,5.5,52.1,2574.2,41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4,High,0.111316,2
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.6,26.7,69.2,11092.6,1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4,"11,041.30",159.4,"11,041.30",159.4,"11,041.30",159.4,"10,859

In [53]:
df_Jan.groupby('Size_Index').size()

Size_Index
1    286
2    285
3    285
4    285
5    285
dtype: int64

In [54]:
### generate the categories further by quintile of BM Ratio within each Size group. 
df_Jan['BM_Ratio_Index'] = df_Jan.groupby(['Size_Index'])['BM_Ratio'].transform(
                     lambda x: pd.qcut(x, 5, labels=np.arange(1,6,1)))

In [55]:
df_Jan.groupby(by=['Size_Index','BM_Ratio_Index']).size()

Size_Index  BM_Ratio_Index
1           1.0               58
            2.0               57
            3.0               57
            4.0               57
            5.0               57
2           1.0               58
            2.0               56
            3.0               57
            4.0               57
            5.0               57
3           1.0               57
            2.0               57
            3.0               57
            4.0               57
            5.0               57
4           1.0               57
            2.0               57
            3.0               57
            4.0               57
            5.0               57
5           1.0               57
            2.0               57
            3.0               57
            4.0               57
            5.0               57
dtype: int64

In [56]:
### generate the categories further by quintile of Momentum within each previous group
df_Jan['Momentum_Index'] = df_Jan.groupby(by=['Size_Index','BM_Ratio_Index'])['Momentum'].transform(
                     lambda x: pd.qcut(x, 5, labels=np.arange(1,6,1)))

In [57]:
df_Jan.groupby(by=['Size_Index','BM_Ratio_Index','Momentum_Index']).size()

Size_Index  BM_Ratio_Index  Momentum_Index
1           1.0             1.0               12
                            2.0               11
                            3.0               12
                            4.0               11
                            5.0               12
            2.0             1.0               12
                            2.0               11
                            3.0               11
                            4.0               11
                            5.0               12
            3.0             1.0               12
                            2.0               11
                            3.0               11
                            4.0               11
                            5.0               12
            4.0             1.0               12
                            2.0               11
                            3.0               11
                            4.0               11
                          

In [58]:
df_Jan_Overview=df_Jan[['Size_Index','BM_Ratio_Index','Momentum_Index','ESG_Index','Monthly Return']]

In [59]:
df_Jan_Overview.groupby(by=['Size_Index','BM_Ratio_Index','Momentum_Index'],group_keys=True,sort=True,as_index=False).size()

,Size_Index,BM_Ratio_Index,Momentum_Index,size
0,1,1.0,1.0,12
1,1,1.0,2.0,11
2,1,1.0,3.0,12
3,1,1.0,4.0,11
4,1,1.0,5.0,12
5,1,2.0,1.0,12
6,1,2.0,2.0,11
7,1,2.0,3.0,11
8,1,2.0,4.0,11
9,1,2.0,5.0,12


In [60]:
df_Jan_Overview_Benchmark = df_Jan_Overview.groupby(by=['Size_Index','BM_Ratio_Index','Momentum_Index'],group_keys=True,sort=True,as_index=False).mean('Monthly Return')

In [61]:
df_Jan_Overview_Benchmark.head()

,Size_Index,BM_Ratio_Index,Momentum_Index,Monthly Return
0,1,1.0,1.0,0.650000
1,1,1.0,2.0,-2.354545
2,1,1.0,3.0,-9.558333
3,1,1.0,4.0,-0.236364
4,1,1.0,5.0,-2.058333


In [62]:
df_Jan_Overview_ESG_High = df_Jan_Overview[df_Jan_Overview['ESG_Index']=='High']
df_Jan_Overview_ESG_High.head()

,Size_Index,BM_Ratio_Index,Momentum_Index,ESG_Index,Monthly Return
A,5,3.0,2.0,High,-3.2
AAN,3,3.0,3.0,High,3.9
AAON,2,2.0,4.0,High,6.1
AAPL,5,3.0,5.0,High,5.4
AAXN,3,3.0,5.0,High,4.8


In [63]:
df_Jan_Overveiw_ESG_High_Mean = df_Jan_Overview_ESG_High.groupby(by=['Size_Index','BM_Ratio_Index','Momentum_Index'],group_keys=True,sort=True,as_index=False).mean('Monthly Return')

In [64]:
df_Jan_Overveiw_ESG_High_Mean.head()

,Size_Index,BM_Ratio_Index,Momentum_Index,Monthly Return
0,1,1.0,1.0,6.766667e+00
1,1,1.0,2.0,-1.900000e+00
2,1,1.0,3.0,-1.050000e+01
3,1,1.0,4.0,-1.110223e-16
4,1,1.0,5.0,-9.800000e+00


In [65]:
df_Jan_Overview_Benchmark['ESG_High_Return']= df_Jan_Overveiw_ESG_High_Mean['Monthly Return']
df_Jan_Overview_Benchmark.head()

,Size_Index,BM_Ratio_Index,Momentum_Index,Monthly Return,ESG_High_Return
0,1,1.0,1.0,0.650000,6.766667e+00
1,1,1.0,2.0,-2.354545,-1.900000e+00
2,1,1.0,3.0,-9.558333,-1.050000e+01
3,1,1.0,4.0,-0.236364,-1.110223e-16
4,1,1.0,5.0,-2.058333,-9.800000e+00


In [66]:
df_Jan_Overview_Benchmark['Abnormal Return']=df_Jan_Overview_Benchmark['ESG_High_Return']-df_Jan_Overview_Benchmark['Monthly Return']
df_Jan_Overview_Benchmark.head()

,Size_Index,BM_Ratio_Index,Momentum_Index,Monthly Return,ESG_High_Return,Abnormal Return
0,1,1.0,1.0,0.650000,6.766667e+00,6.116667
1,1,1.0,2.0,-2.354545,-1.900000e+00,0.454545
2,1,1.0,3.0,-9.558333,-1.050000e+01,-0.941667
3,1,1.0,4.0,-0.236364,-1.110223e-16,0.236364
4,1,1.0,5.0,-2.058333,-9.800000e+00,-7.741667


In [67]:
df_Jan_Overview_Benchmark['Month']='Jan'
df_Jan_Overview_Benchmark.head()

,Size_Index,BM_Ratio_Index,Momentum_Index,Monthly Return,ESG_High_Return,Abnormal Return,Month
0,1,1.0,1.0,0.650000,6.766667e+00,6.116667,Jan
1,1,1.0,2.0,-2.354545,-1.900000e+00,0.454545,Jan
2,1,1.0,3.0,-9.558333,-1.050000e+01,-0.941667,Jan
3,1,1.0,4.0,-0.236364,-1.110223e-16,0.236364,Jan
4,1,1.0,5.0,-2.058333,-9.800000e+00,-7.741667,Jan


In [68]:
import plotly.express as px

In [69]:
fig = px.histogram(df_Jan_Overview_Benchmark, x="Abnormal Return",
                   hover_data=df_Jan_Overview_Benchmark.columns)
fig.show()

In [70]:
df_Jan_Overview_Benchmark.to_csv('Overview_Jan.csv')

In [71]:
import plotly.graph_objects as go

In [72]:
fig = go.Figure()
fig.add_trace(go.Box(
    y=df_Jan_Overview_Benchmark['Abnormal Return'],
    boxmean=True # represent mean
))

In [73]:
### Save unaggregated data
df_Jan.head()

,Name,Stock Exchange,Company Main Business,Size,Book Value Per Shr Tangble,Common Shares Outstdg,MktVal Co,Momentum,Monthly Return,Sales,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,1 MktVal Co,1 Closing Price,2 MktVal Co,2 Closing Price,3 MktVal Co,3 Closing Price,4 MktVal Co,4 Closing Price,5 MktVal Co,5 Closing Price,6 MktVal Co,6 Closing Price,7 MktVal Co,7 Closing Price,8 MktVal Co,8 Closing Price,9 MktVal Co,9 Closing Price,10 MktVal Co,10 Closing Price,11 MktVal Co,11 Closing Price,12 MktVal Co,12 Closing Price,13 MktVal Co,13 Closing Price,14 MktVal Co,14 Closing Price,15 MktVal Co,15 Closing Price,16 MktVal Co,16 Closing Price,17 MktVal Co,17 Closing Price,18 MktVal Co,18 Closing Price,19 MktVal Co,19 Closing Price,20 MktVal Co,20 Closing Price,21 MktVal Co,21 Closing Price,22 MktVal Co,22 Closing Price,23 MktVal Co,23 Closing Price,24 MktVal Co,24 Closing Price,25 MktVal Co,25 Closing Price,26 MktVal Co,26 Closing Price,27 MktVal Co,27 Closing Price,28 MktVal Co,28 Closing Price,29 MktVal Co,29 Closing Price,30 MktVal Co,30 Closing Price,31 MktVal Co,31 Closing Price,STD MktVal,STD Closing Price,ESG_Index,BM_Ratio,Size_Index,BM_Ratio_Index,Momentum_Index
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.7,0.5,309.7,26461.7,27.5,-3.2,"5,223.00",10.0,5.5,6.6,AAA,"26,461.70",85.3,"26,660.20",86,"26,232.20",84.6,"26,232.20",84.6,"26,232.20",84.6,"26,309.70",84.8,"26,390.40",85.1,"26,650.90",85.9,"27,069.70",87.3,"27,168.90",87.6,"27,168.90",87.6,"27,168.90",87.6,"27,128.60",87.5,"27,293.00",88,"27,488.40",88.6,"27,749.00",89.5,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,953.70",90.1,"27,795.50",89.6,"27,956.80",90.1,"27,628.00",89.1,"27,376.80",88.3,"27,376.80",88.3,"27,376.80",88.3,"26,849.40",86.6,"27,041.80",87.2,"27,060.40",87.2,"26,173.20",84.4,"25,608.70",82.6,647.9,2.1,High,0.005852,5,3.0,2.0
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.8,-14.7,428.2,12280.8,-9.4,-6.4,"33,361.00",3.3,5.1,2.5,CCC,"12,280.80",28.7,"12,456.30",29.1,"11,839.70",27.7,"11,839.70",27.7,"11,839.70",27.7,"11,698.40",27.3,"11,655.60",27.2,"11,921.10",27.8,"11,968.20",28,"11,698.40",27.3,"11,698.40",27.3,"11,698.40",27.3,"11,728.40",27.4,"11,788.40",27.5,"11,809.80",27.6,"12,088.10",28.2,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"12,160.90",28.4,"11,647.00",27.2,"11,698.40",27.3,"12,332.20",28.8,"11,835.50",27.6,"11,835.50",27.6,"11,835.50",27.6,"11,180.30",26.1,"11,518.60",26.9,"11,475.80",26.8,"11,869.70",27.7,"11,492.90",26.8,276.2,0.6,Low,-0.512551,4,1.0,1.0
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.1,12.1,66.7,3835.1,36.2,3.9,"4,098.80",3.4,6.0,4.8,A,"3,835.10",57.1,"3,872.70",57.7,"3,825.70",57,"3,825.70",57,"3,825.70",57,"3,873.30",57.7,"3,867.30",57.6,"3,935.10",58.6,"3,888.10",57.9,"3,876.00",57.7,"3,876.00",57.7,"3,876.00",57.7,"3,913.60",58.3,"3,963.30",59,"4,006.30",59.7,"4,045.20",60.2,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,121.80",61.4,"4,054.00",60.4,"4,104.30",61.1,"4,120.40",61.4,"4,128.50",61.5,"4,128.50",61.5,"4,128.50",61.5,"4,125.80",61.4,"4,215.10",62.8,"4,278.90",63.7,"4,217.10",62.8,"3,986.10",59.4,137.1,2.0,High,0.210443,3,3.0,3.0
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.2,5.5,52.1,2574.2,41.8,6.1,499.2,5.5,7.0,8.4,A,"2,574.20",49.4,"2,599.20",49.9,"2,623.10",50.4,"2,623.10",50.4,"2,623.10",50.4,"2,618.40",50.3,"2,612.70",50.2,"2,605.40",50,"2,649.20",50.9,"2,647.10",50.8,"2,647.10",50.8,"2,647.10",50.8,"2,706.50",52,"2,704.10",51.9,"2,723.70",52.3,"2,745.60",52.7,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,754.90",52.9,"2,753.40",52.9,"2,758.60",53,"2,802.90",53.8,"2,784.60",53.5,"2,784.60",53.5,"2,784.60",53.5,"2,782.00",53.4,"2,797.10",53.7,"2,793.00",53.6,"2,827.90",54.3,"2,732.00",52.4,75.5,1.4,High,0.111316,2,2.0,4.0
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.6,26.7,69.2,11092.6,1.9,-17.7,"9,624.00",2.0,5.9,4.6,BBB,"11,092.60",160.2,"11,040.60",159.4

In [74]:
df_Jan_Return = df_Jan[['Name','Stock Exchange','Company Main Business','Size','BM_Ratio','Momentum','MSCI ESG Rtg Score Environmental','MSCI ESG Rtg Score Governance','MSCI ESG Rtg Score Social','MSCI ESG Rtg Overall Company Rating','Size_Index','BM_Ratio_Index','Momentum_Index','ESG_Index','Monthly Return']]

In [75]:
df_Jan_Return['Month']='Jan'
df_Jan_Return['Symbol']=df_Jan_Return.index
df_Jan_Return.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Name,Stock Exchange,Company Main Business,Size,BM_Ratio,Momentum,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Governance,MSCI ESG Rtg Score Social,MSCI ESG Rtg Overall Company Rating,Size_Index,BM_Ratio_Index,Momentum_Index,ESG_Index,Monthly Return,Month,Symbol
A,"Agilent Technologies, Inc.",NYSE,Healthcare,26461.7,0.005852,27.5,10.0,5.5,6.6,AAA,5,3.0,2.0,High,-3.2,Jan,A
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,12280.8,-0.512551,-9.4,3.3,5.1,2.5,CCC,4,1.0,1.0,Low,-6.4,Jan,AAL
AAN,"Aaron's, Inc.",NYSE,Consumer Cyclicals,3835.1,0.210443,36.2,3.4,6.0,4.8,A,3,3.0,3.0,High,3.9,Jan,AAN
AAON,"AAON, Inc.",NASDAQ,Industrials,2574.2,0.111316,41.8,5.5,7.0,8.4,A,2,2.0,4.0,High,6.1,Jan,AAON
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,11092.6,0.166565,1.9,2.0,5.9,4.6,BBB,4,3.0,1.0,Low,-17.7,Jan,AAP


In [76]:
df_Jan_Return.to_csv('Return_Jan')